In [6]:
from functools import partial
import sections
import text_reader as tp
from buffered_iterator import BufferedIterator

In [7]:
DVH_TEST_TEXT = [
        'Patient Name         : ____, ____',
        'Patient ID           : 1234567',
        'Comment              : DVHs for multiple plans and plan sums',
        'Date                 : Friday, January 17, 2020 09:45:07',
        'Exported by          : gsal',
        'Type                 : Cumulative Dose Volume Histogram',
        ('Description          : The cumulative DVH displays the '
        'percentage (relative)'),
        ('                       or volume (absolute) of structures '
        'that receive a dose'),
        '                      equal to or greater than a given dose.',
        '',
        'Plan sum: Plan Sum',
        'Course: PLAN SUM',
        'Prescribed dose [cGy]: not defined',
        '% for dose (%): not defined',
        '',
        'Plan: PARR',
        'Course: C1',
        'Plan Status: Treatment Approved Thursday, January 02, '
        '2020 12:55:56 by gsal',
        'Prescribed dose [cGy]: 5000.0',
        '% for dose (%): 100.0',
        '',
        'Structure: PRV5 SpinalCanal',
        'Approval Status: Approved',
        'Plan: Plan Sum',
        'Course: PLAN SUM',
        'Volume [cm³]: 121.5',
        'Dose Cover.[%]: 100.0',
        'Sampling Cover.[%]: 100.1',
        'Min Dose [cGy]: 36.7',
        'Max Dose [cGy]: 3670.1',
        'Mean Dose [cGy]: 891.9',
        'Modal Dose [cGy]: 44.5',
        'Median Dose [cGy]: 863.2',
        'STD [cGy]: 621.9',
        'NDR: ',
        'Equiv. Sphere Diam. [cm]: 6.1',
        'Conformity Index: N/A',
        'Gradient Measure [cm]: N/A',
        '',
        'Dose [cGy] Ratio of Total Structure Volume [%]',
        '         0                       100',
        '         1                       100',
        '         2                       100',
        '         3                       100',
        '         4                       100',
        '         5                       100',
        '      3667              4.23876e-005',
        '      3668              2.87336e-005',
        '      3669              1.50797e-005',
        '      3670               1.4257e-006',
        '',
        'Structure: PTV 50',
        'Approval Status: Approved',
        'Plan: Plan Sum',
        'Course: PLAN SUM',
        'Volume [cm³]: 363.6',
        'Dose Cover.[%]: 100.0',
        'Sampling Cover.[%]: 100.0',
        'Min Dose [cGy]: 3985.9',
        'Max Dose [cGy]: 5442.0',
        'Mean Dose [cGy]: 5144.5',
        'Modal Dose [cGy]: 5177.3',
        'Median Dose [cGy]: 5166.9',
        'STD [cGy]: 131.9',
        'NDR: ',
        'Equiv. Sphere Diam. [cm]: 8.9',
        'Conformity Index: N/A',
        'Gradient Measure [cm]: N/A',
        '',
        'Dose [cGy] Ratio of Total Structure Volume [%]',
        '         0                       100',
        '         1                       100',
        '         2                       100',
        '         3                       100',
        '         4                       100',
        '         5                       100',
        '      5437               9.4777e-005',
        '      5438              6.35607e-005',
        '      5439              3.62425e-005',
        '      5440              1.82336e-005',
        '      5441              9.15003e-006',
        '      5442               6.6481e-008'
        ]


In [8]:
context = {}

dvh_info_end = sections.SectionBreak(
    name='End of DVH Info',
    sentinel=['Plan:', 'Plan sum:']
    )

dvh_info_section = sections.Section(
    start_section=None,
    end_section=dvh_info_end
    )


## Using `Scan`.

### Identifying first source item:

- Use `is_first_item` attribute to check for first line of source.
  - Values can be:
  > 1. None $\to$ Before first item
  > 2. True $\to$ Is first item
  > 3. False $\to$ After first item
- When calling next item:
```python
if self.is_first_item is None:
     self.is_first_item = True
else:
     self.is_first_item = False
```

### Other useful values
- Source count at start of section (0) if Source is new.
- Source count after _Move to Start_
  
### Section vs. Source Count
- Section count should match with Source cound because scan returns all source 
  items between start and end of section.
  - `scan` is not called by `process` or `read`.
  - `self.source_index` can be set in `scan`.

**********
```python
  self._source_index = None  # clear the indexing
  
  @property
  def source_index(self):
      return self._source_index

  @property
  def source_item_count(self):
      if not self._source_index:
          return 0
      return self._source_index[-1]

  @property
  def item_count(self):
      if not self._source_index:
          return 0
      return len(self._source_index)
```
**********


In [9]:
end_check = dvh_info_section.scan(DVH_TEST_TEXT, start_search=True)
output = [row for row in end_check]  # pylint: disable=unused-variable
event = dvh_info_section.context['Event']
print(event)

Plan sum:


In [10]:
end_check = dvh_info_section.scan(DVH_TEST_TEXT, start_search=True)
scanned_lines = [row for row in end_check]
print(scanned_lines)

['Patient Name         : ____, ____', 'Patient ID           : 1234567', 'Comment              : DVHs for multiple plans and plan sums', 'Date                 : Friday, January 17, 2020 09:45:07', 'Exported by          : gsal', 'Type                 : Cumulative Dose Volume Histogram', 'Description          : The cumulative DVH displays the percentage (relative)', '                       or volume (absolute) of structures that receive a dose', '                      equal to or greater than a given dose.', '']
